# ⬇️ Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pathlib import Path


import warnings
warnings.filterwarnings('ignore')


# ✅ Downloading the data

region (department) exceptions :
- 20 not existing ->2A and 2B
- 57, 67, 68 -> no data
- 69 -> 69D and 59M

In [2]:
## list all the number of regions we can download
regions = list(np.arange(1,96))
# regions = list(map(str, regions))
regions = list(np.delete(regions, [20-1, 57-1, 67-1, 68-1]))
regions = regions + ['2A', '2B']
print(regions)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, '2A', '2B']


In [3]:
# download all csv files

# from pathlib import Path

for index, num in enumerate(regions):
    if index < 9 :
        url = f'https://dvf-api.data.gouv.fr/dvf/csv/?dep=0{num}'
    else :
        url = f'https://dvf-api.data.gouv.fr/dvf/csv/?dep={num}'

    path = Path(f'../raw_data/dvf_{num}.csv')

    if not path.is_file():
        df = pd.read_csv(url)
        df.to_csv(path)
        print(f'Downloaded region n° {num} 💪')

    else:
        print(f'region n° {num} is already downloaded ! 🚀')


region n° 1 is already downloaded ! 🚀
region n° 2 is already downloaded ! 🚀
region n° 3 is already downloaded ! 🚀
region n° 4 is already downloaded ! 🚀
region n° 5 is already downloaded ! 🚀
region n° 6 is already downloaded ! 🚀
region n° 7 is already downloaded ! 🚀
region n° 8 is already downloaded ! 🚀
region n° 9 is already downloaded ! 🚀
region n° 10 is already downloaded ! 🚀
region n° 11 is already downloaded ! 🚀
region n° 12 is already downloaded ! 🚀
region n° 13 is already downloaded ! 🚀
region n° 14 is already downloaded ! 🚀
region n° 15 is already downloaded ! 🚀
region n° 16 is already downloaded ! 🚀
region n° 17 is already downloaded ! 🚀
region n° 18 is already downloaded ! 🚀
region n° 19 is already downloaded ! 🚀
region n° 21 is already downloaded ! 🚀
region n° 22 is already downloaded ! 🚀
region n° 23 is already downloaded ! 🚀
region n° 24 is already downloaded ! 🚀
region n° 25 is already downloaded ! 🚀
region n° 26 is already downloaded ! 🚀
region n° 27 is already downloaded

In [1]:
# # concat all csv files in 1 big dataframe --> too big for the kernel
# df = pd.DataFrame()
# for num in regions:
#     df = pd.concat([df, pd.read_csv(f'../raw_data/dvf_{num}.csv')])

In [ ]:
# ### save full df raw as 1 csv
# df.to_csv(f'../raw_data/dvf_full_raw.csv')

In [ ]:
# df.shape

In [ ]:
# df_dvf = pd.read_csv('../raw_data/dvf_93.csv')

# df_dvf.shape

In [ ]:
df_dvf['code_departement']

# 🔎 Exploring

In [ ]:
df_dvf.duplicated().sum()

In [ ]:
df_dvf['nature_mutation'].unique()

In [ ]:
df_dvf['nature_mutation'].value_counts()

In [ ]:
df_dvf.dtypes

In [ ]:
df_dvf.columns

In [ ]:
df_dvf.isnull().sum()

In [ ]:
df_dvf[['code_postal', 'code_commune', 'nom_commune']].isnull().sum()/len(df_dvf)

In [ ]:
df_dvf['nombre_lots'].astype('str').value_counts()

In [ ]:
df_dvf['type_local'].unique()

# 🔅 Preprocessing

## 🧹 Keep only useful columns and rows and translate

### Removing columns

In [ ]:
keep_col =['date_mutation',
            'nature_mutation',
            'valeur_fonciere',
            'code_postal',
            'code_commune',
            'code_departement',
            'nombre_lots',
            'type_local',
            'surface_reelle_bati',
            'nombre_pieces_principales',
            'longitude',
            'latitude']
df_dvf = df_dvf[keep_col]

df_dvf.info()

### Translate column-names

In [ ]:
df_dvf.columns = ['date', 'built', 'price', 'postal_code',
                'city', 'region', 'number_of_units', 'property_type',
                'living_area', 'number_of_rooms',
                'longitude', 'latitude']

### Remove non-representative rows

In [ ]:
df_dvf.shape

In [ ]:
df_dvf['built'].unique()

In [ ]:
df_dvf.shape

In [ ]:
df_dvf['built'].unique()

We want to keep only :
- Houses and appartments
- Normal sales and off-plan sales
- Sales with only one unit

In [ ]:
df_dvf['built'].unique()

In [ ]:
print('Shape before the masks : ', df_dvf.shape)
print('Shape only normal sales', df_dvf[((df_dvf['built'] == "Vente") | (df_dvf['built'] == "Vente en l'état futur d'achèvement"))].shape)
print('Shape only 1 unit', df_dvf[((df_dvf['number_of_units'] == 1) | (df_dvf['number_of_units'] == '1'))].shape)
print('Shape only house and appartments', df_dvf[((df_dvf['property_type'] == 'Appartement') | (df_dvf['property_type'] == 'Maison'))].shape)

In [ ]:
df_useful = df_dvf[((df_dvf['built'] == "Vente") | (df_dvf['built'] == "Vente en l'état futur d'achèvement")) &
                    ((df_dvf['number_of_units'] == 1) | (df_dvf['number_of_units'] == '1')) &
                    ((df_dvf['property_type'] == 'Appartement') | (df_dvf['property_type'] == 'Maison'))]
df_useful.shape

In [ ]:
## checking that the masks worked correctly
print(df_useful['built'].unique())
print(df_useful['number_of_units'].unique())
print(df_useful['property_type'].unique())


### Translating the values

In [ ]:
trans_dict_built = {'Vente' : 'built',
                    'Vente en l’état futur d’achèvement' : 'off-plan'}
trans_dict_type = {'Appartement' : 'appartment',
                   'Maison' : 'house'}
df_useful= df_useful.replace({'built' : trans_dict_built,
                             'property type' : trans_dict_type})

In [ ]:
# checking we didn't lose rows
df_useful.shape

## Dropping duplicates and NaNs

In [ ]:
df_useful.columns

In [ ]:
## dropping the column with number of units (only 1s)
df_useful = df_useful.drop(columns='number_of_units')

In [ ]:
df_useful.columns

In [ ]:
## checking for NaN values
df_useful.isna().sum()
df_useful = df_useful.dropna()
df_useful.shape
df_useful = df_useful.drop_duplicates()
df_useful.shape

## Changing everything to the right type

In [ ]:
col_float = ['price', 'longitude', 'latitude', 'postal_code', 'living_area', 'number_of_rooms']
col_string = ['built','city', 'region','property_type']
col_date = ['date']
### Formating data types ####
df_useful[col_float] = df_useful[col_float].apply(lambda x: pd.to_numeric(x, errors='coerce').astype('float64'))
df_useful[col_date]= df_useful[col_date].apply(lambda x: pd.to_datetime(x, errors='coerce'))

df_useful.info()

In [ ]:
df_useful.shape

In [ ]:
df_useful.head()

In [ ]:
# import folium
# m = folium.Map(location = [-33.8, 151.2], tiles ='OpenStreetMap', zoom_start=11)
import seaborn as sns
sns.scatterplot(x = df_useful['longitude'], y = df_useful['latitude'])

## 🤓 Feature engineering

In [ ]:
df_dvf['prix_m_2'] = df_dvf['valeur_fonciere']/ df_dvf['surface_reelle_bati']



df_dvf.head(5)

In [ ]:
df_dvf['surface_terrain'].fillna(value=0, inplace=True)

df_dvf.head(5)

In [ ]:
df_dvf.info()

In [ ]:
df_dvf.head(5)

In [ ]:
import project

In [ ]:
df_dvf